In [1]:
!pip install torch torchvision kaggle


In [20]:
# After uploading your kaggle.json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download the dataset
!kaggle datasets download -d rkuo2000/uecfood100

# Unzip the dataset
!unzip uecfood100.zip -d ./food_dataset


cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/rkuo2000/uecfood100
License(s): unknown
uecfood100.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  uecfood100.zip
replace ./food_dataset/UECFOOD100/1/1.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [21]:
from torchvision.datasets import ImageFolder

class SortedImageFolder(ImageFolder):
    def __init__(self, root, transform=None):
        super(SortedImageFolder, self).__init__(root, transform)
        # Override class_to_idx to sort folders by numeric order
        self.classes = sorted(self.classes, key=lambda x: int(x))  # Sorting folder names numerically
        self.class_to_idx = {cls_name: i for i, cls_name in enumerate(self.classes)}  # Rebuild class_to_idx
        self.samples = sorted(self.samples, key=lambda x: (self.class_to_idx[os.path.basename(os.path.dirname(x[0]))], x[0]))

In [22]:
import os
import torch
from torch import nn, optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import os

# Set paths to train and validation directories
train_data_dir = './food_dataset/UECFOOD100'
val_data_dir = './food_dataset/UECFOOD100'  # Assuming validation set exists


In [23]:
folders = os.listdir(train_data_dir)
sorted_folders = sorted([folder for folder in folders if folder.isdigit()], key=int)

print(sorted_folders)

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100']


In [24]:
# Define transformations for training and validation sets
train_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the dataset
train_dataset = SortedImageFolder(root=train_data_dir, transform=train_transforms)
val_dataset = SortedImageFolder(root=val_data_dir, transform=val_transforms)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


In [25]:
print(train_dataset.class_to_idx)

{'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7, '9': 8, '10': 9, '11': 10, '12': 11, '13': 12, '14': 13, '15': 14, '16': 15, '17': 16, '18': 17, '19': 18, '20': 19, '21': 20, '22': 21, '23': 22, '24': 23, '25': 24, '26': 25, '27': 26, '28': 27, '29': 28, '30': 29, '31': 30, '32': 31, '33': 32, '34': 33, '35': 34, '36': 35, '37': 36, '38': 37, '39': 38, '40': 39, '41': 40, '42': 41, '43': 42, '44': 43, '45': 44, '46': 45, '47': 46, '48': 47, '49': 48, '50': 49, '51': 50, '52': 51, '53': 52, '54': 53, '55': 54, '56': 55, '57': 56, '58': 57, '59': 58, '60': 59, '61': 60, '62': 61, '63': 62, '64': 63, '65': 64, '66': 65, '67': 66, '68': 67, '69': 68, '70': 69, '71': 70, '72': 71, '73': 72, '74': 73, '75': 74, '76': 75, '77': 76, '78': 77, '79': 78, '80': 79, '81': 80, '82': 81, '83': 82, '84': 83, '85': 84, '86': 85, '87': 86, '88': 87, '89': 88, '90': 89, '91': 90, '92': 91, '93': 92, '94': 93, '95': 94, '96': 95, '97': 96, '98': 97, '99': 98, '100': 99}


In [26]:
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load pre-trained ResNet-50 model
model = models.resnet50(pretrained=True)

# Freeze all layers except the last fully connected layer
for param in model.parameters():
    param.requires_grad = False

# Replace the fully connected layer to classify 100 classes (for UEC-Food100)
num_classes = 100
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Move the model to GPU if available
model = model.to(device)


In [ ]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

# Training loop
epochs = 10  # You can adjust this based on your time constraints

for epoch in range(epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss / len(train_loader)}")

    # Validation step
    model.eval()  # Set model to evaluation mode
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Validation Loss: {val_loss / len(val_loader)}, Accuracy: {100 * correct / total:.2f}%")

# Save the trained model
model_save_path = 'food_resnet50_colab.pth'
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")


Epoch [1/10], Loss: 3.0803133182907954
Validation Loss: 1.9184987564594018, Accuracy: 52.61%
Epoch [2/10], Loss: 2.240920371095959
Validation Loss: 1.6243623299296024, Accuracy: 58.14%
Epoch [3/10], Loss: 2.0507322330782833
Validation Loss: 1.4677556711342392, Accuracy: 60.48%
Epoch [4/10], Loss: 1.9756266571631145
Validation Loss: 1.435230767624017, Accuracy: 61.61%
Epoch [5/10], Loss: 1.896207748914349
Validation Loss: 1.4170725752697027, Accuracy: 60.88%
Epoch [6/10], Loss: 1.8907678960957346
Validation Loss: 1.3196811223016816, Accuracy: 64.15%
Epoch [7/10], Loss: 1.8107701291751224
Validation Loss: 1.2498985037709565, Accuracy: 65.63%


In [14]:
from google.colab import files
files.download('food_resnet50_colab.pth')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>